In [1]:
import uproot
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os #for looping over files in a directory
import math
import json
import glob

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (np.integer,)):
            return int(obj)
        elif isinstance(obj, (np.floating,)):
            return float(obj)
        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()
        return super().default(obj)

def load_json_file(file_path):
    """
    Loads JSON data from a file.

    Args:
        file_path (str): The path to the JSON file.

    Returns:
        dict or list: A Python dictionary or list representing the JSON data, or None if an error occurs.
    """
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
            return data
    except FileNotFoundError:
        print(f"Error: File not found at '{file_path}'")
        return None
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in '{file_path}'")
        return None
    except Exception as e:
         print(f"An unexpected error occurred: {e}")
         return None

In [2]:
# some cuts, like mindphijetmet, nbjet=0, nlepton=0, already applied at preselection level for now
SRcuts={'met_met': 200,
        'mTGammaMet': 50,
        'j1_pt': 150,
        'ph_pt': 10,
        'met_signif': 25}

def getemptyresults():
    results={}
    for b in ['TT', 'TL', 'LT', 'LL']:
        results[b] = {'real': {'nevents': 0,
                               'sumweights': 0},
                      'jfp': {'nevents': 0,
                               'sumweights': 0},
                      'efp': {'nevents': 0,
                               'sumweights': 0},
                      'other': {'nevents': 0,
                                'sumweights': 0},
                      'unclassified': {'nevents': 0,
                                       'sumweights': 0},
                      'data': 0
                     }
    return results

def ABCDresults(data,mask,isMC,ID="tightID",Iso="tightIso"):
    masks={}
    if Iso == "hybridIso":
        selhybridIsoMask = ((data['ph_pt']>20000) & (data['ph_select_looseIso']==1)) | ((data['ph_pt']<20000) & (data['ph_select_tightIso']==1))
        rejhybridIsoMask = ((data['ph_pt']>20000) & (data['ph_select_looseIso']==0)) | ((data['ph_pt']<20000) & (data['ph_select_tightIso']==0))
        masks['TT'] = (data[f'ph_select_{ID}']==1) & selhybridIsoMask & ((data['ph_isEM'] & 0x45fc01)==0)
        masks['TL'] = (data[f'ph_select_{ID}']==1) & rejhybridIsoMask & ((data['ph_isEM'] & 0x45fc01)==0)
        masks['LT'] = (data[f'ph_select_{ID}']==0) & selhybridIsoMask & ((data['ph_isEM'] & 0x45fc01)==0)
        masks['LL'] = (data[f'ph_select_{ID}']==0) & rejhybridIsoMask & ((data['ph_isEM'] & 0x45fc01)==0)
    elif Iso == "noIso":
        masks['TT'] = (data[f'ph_select_{ID}']==1)
        masks['TL'] = (0==1)
        masks['LT'] = (data[f'ph_select_{ID}']==0) & ((data['ph_isEM'] & 0x45fc01)==0)
        masks['LL'] = (0==1)
        
    else:
        masks['TT'] = (data[f'ph_select_{ID}']==1) & (data[f'ph_select_{Iso}']==1) & ((data['ph_isEM'] & 0x45fc01)==0)
        masks['TL'] = (data[f'ph_select_{ID}']==1) & (data[f'ph_select_{Iso}']==0) & ((data['ph_isEM'] & 0x45fc01)==0)
        masks['LT'] = (data[f'ph_select_{ID}']==0) & (data[f'ph_select_{Iso}']==1) & ((data['ph_isEM'] & 0x45fc01)==0)
        masks['LL'] = (data[f'ph_select_{ID}']==0) & (data[f'ph_select_{Iso}']==0) & ((data['ph_isEM'] & 0x45fc01)==0)

    if isMC:
        real_mask  = (data['ph_truthprompt'] == 1)
        jfp_mask   = (data['ph_truthJFP']    == 1)
        efp_mask   = (data['ph_truthEFP']    == 1)
        other_mask = (data['ph_truthother']  == 1)

    results=getemptyresults()
    for b in ['TT', 'TL', 'LT', 'LL']:
        if isMC:
            results[b]['real']['nevents']            = np.sum(                     mask & masks[b] & real_mask)
            results[b]['real']['sumweights']         = np.sum(data['weight_total'][mask & masks[b] & real_mask])
            results[b]['jfp']['nevents']             = np.sum(                     mask & masks[b] & jfp_mask)
            results[b]['jfp']['sumweights']          = np.sum(data['weight_total'][mask & masks[b] & jfp_mask])
            results[b]['efp']['nevents']             = np.sum(                     mask & masks[b] & efp_mask)
            results[b]['efp']['sumweights']          = np.sum(data['weight_total'][mask & masks[b] & efp_mask])
            results[b]['other']['nevents']           = np.sum(                     mask & masks[b] & other_mask)
            results[b]['other']['sumweights']        = np.sum(data['weight_total'][mask & masks[b] & other_mask])
            results[b]['unclassified']['nevents']    = np.sum(                     mask & masks[b] & ~real_mask & ~jfp_mask & ~efp_mask & ~other_mask)
            results[b]['unclassified']['sumweights'] = np.sum(data['weight_total'][mask & masks[b] & ~real_mask & ~jfp_mask & ~efp_mask & ~other_mask])
        else:
            results[b]['data'] = np.sum(mask & masks[b])
    return results

def dumpjson(data,isMC,ID="tightID",Iso="tightIso"):

    PS={}
    PS['0L'] = \
    (data['met_met']         >  SRcuts['met_met']*1000.   ) & \
    (data['jet_cleanTightBad_prod'] == 1                  ) & \
    (data['j1_pt']           >  SRcuts['j1_pt']*1000.     ) & \
    (data['ph_pt']           >  SRcuts['ph_pt']*1000.     ) & \
    (data['mindPhiJetMet']   >  0.4                       ) & \
    (data['nBTagJets']       == 0                         ) & \
    (data['nElectrons']      == 0                         ) & \
    (data['nMuons']          == 0                         ) & \
    (data['nTau20_baseline']        == 0                  )

    
    SR={}
    SR['0L-mT-low'] = PS['0L'] & \
    (data['mTGammaMet']      <  50.*1000.) & \
    (data['met_signif']      >  25       ) & \
    (data['mindPhiGammaJet'] >  1.5      )

    SR['0L-mT-mid'] = PS['0L'] & \
    (data['mTGammaMet']      >   50*1000.) & \
    (data['mTGammaMet']      <  115*1000.) & \
    (data['met_signif']      >  20       ) & \
    (data['mindPhiGammaJet'] >  1.5      ) & \
    (data['dPhiGammaJ1']     >  1.5       )

    SR['0L-mT-hgh'] = PS['0L'] & \
    (data['mTGammaMet']      >  115*1000.) & \
    (data['met_signif']      >  15       ) & \
    (data['mindPhiGammaJet'] >  1.5      ) & \
    (data['dPhiGammaJ1']     >  1.5      )

    VR={}
    VR['0L-mT-mid'] = PS['0L'] & \
    (data['mTGammaMet']      >   50*1000.) & \
    (data['mTGammaMet']      <  100*1000.) & \
    (data['dPhiGammaMet']    >  2.0      )
    #(data['dPhiGammaMet']    >  1.5      )
    #(data['mindPhiGammaJet'] <  1.0      )
    
    
    return {'SR': {'0L-mT-low': ABCDresults(data, SR['0L-mT-low'], isMC, ID=ID, Iso=Iso),
                   '0L-mT-mid': ABCDresults(data, SR['0L-mT-mid'], isMC, ID=ID, Iso=Iso),
                   '0L-mT-hgh': ABCDresults(data, SR['0L-mT-hgh'], isMC, ID=ID, Iso=Iso),
                  },
            'VR': {'0L-mT-mid': ABCDresults(data, VR['0L-mT-mid'], isMC, ID=ID, Iso=Iso),
                  },
           }

In [3]:
base_path = "/data/mhance/SUSY/ntuples/v3.1_condor_2"

# Iterate over subdirectories and files
for root, _, files in os.walk(base_path):
    for file in files:
        if not file.endswith('.root'): continue
        if "a.root" in file or "d.root" in file or "e.root" in file: continue
        #if "N2_" in file: continue

        #if not "data_20" in file: continue
        filepath = os.path.join(root, file)

        #if "data_" not in filepath: continue
        #if filepath != "/data/mhance/SUSY/ntuples/v3/output_Wtaunugamma.root": continue
        #if filepath != "/data/mhance/SUSY/ntuples/v3/output_data_2018.root": continue
        #if filepath != "/data/mhance/SUSY/ntuples/v3/output_Znunu_CVetoBVeto.root": continue
        #if filepath != "/data/mhance/SUSY/ntuples/v3/output_N2_220_N1_200_HH.root": continue
        #print(filepath)
        with uproot.open(filepath) as f:
            if 'picontuple' in f:
                tree = f['picontuple']
                # Extract the data
                data = tree.arrays(library="np")
                #data['met_signif'] = data['met_met']/data['ph_pt']

                for ID in ["mediumID", "tightID"]:
                    for Iso in ["looseIso", "tightIso", "tightCOIso", "hybridIso", "noIso","hybridCOIso"]:
                        results=dumpjson(data,"data_" not in filepath, ID, Iso)
                        #print(json.dumps(results, indent=4, cls=NumpyEncoder))
                        with open("ABCD_results/"+file.replace(".root",f"_ABCD_{ID}_{Iso}.json"),'w') as jf:
                            json.dump(results, jf, indent=4, cls=NumpyEncoder)

In [4]:
def getfakeestimate(regiontype="SR",regionname="0L-mT-low",ID="tightID",Iso="tightIso",Run2=True,debug=False):
    
    totalresults=getemptyresults()
    sample_max={}
    sample_max['TL']=[0,'']
    sample_max['LT']=[0,'']
    sample_max['TT']=[0,'']
    sample_max['LL']=[0,'']
    
    samples=[]
    
    for fp in glob.glob(f"ABCD_results/*{ID}_{Iso}.json"):
        if "gammajet" in fp: continue
        if "jetjet" in fp: continue
        if "jj" in fp: continue
        if "N2" in fp: continue

        isRun3 = ("mc23" in fp or "data_202" in fp)
        isRun2 = ("mc20" in fp or "data_201" in fp)
        
        if Run2 and not isRun2: continue # only run on one run at a time
        elif not Run2 and isRun2: continue
            
        data = load_json_file(fp)
    
        sample_tag = fp.replace("_results/output_","").replace("ABCD","").replace(".json","")[:-1]
        samples.append(sample_tag)

        region=data[regiontype][regionname]
        
        for b in ['TT', 'TL', 'LT', 'LL']:
            totalresults[b]['data'] += region[b]["data"]
            totalresults[b]['real']['sumweights'] += region[b]["real"]["sumweights"]
            totalresults[b]['jfp']['sumweights'] += region[b]["jfp"]["sumweights"]
            totalresults[b]['efp']['sumweights'] += region[b]["efp"]["sumweights"]
            totalresults[b]['other']['sumweights'] += region[b]["other"]["sumweights"]
            totalresults[b]['unclassified']['sumweights'] += region[b]["unclassified"]["sumweights"]
    
            if sample_max[b][0] < region[b]["real"]["sumweights"]:
                sample_max[b][0] = region[b]["real"]["sumweights"]
                sample_max[b][1] = sample_tag
                
    if debug:
        print(json.dumps(totalresults,indent=4,cls=NumpyEncoder))

        print("Most contributing samples:")
        for b in ['TT', 'TL', 'LT', 'LL']:
            print(f"{b}: {sample_max[b][1][:-1]}")
        
        mcs_data=load_json_file(f"ABCD_results/output_{sample_max['TT'][1]}_ABCD.json")
        print(json.dumps(mcs_data,indent=4,cls=NumpyEncoder))

    return totalresults,samples

def getsignalestimate(regiontype="SR",regionname="0L-mT-low",ID="tightID",Iso="tightIso",sample="N2_200_N1_190",Run2=True,debug=False):
    signalresults=getemptyresults()
    
    samples=[]
    
    for fp in glob.glob(f"ABCD_results/*{sample}*{ID}_{Iso}.json"):
        data = load_json_file(fp)
    
        isRun3 = ("mc23" in fp or "data_202" in fp)
        isRun2 = ("mc20" in fp or "data_201" in fp)
        
        if Run2 and not isRun2: continue # only run on one run at a time
        elif not Run2 and isRun2: continue

        sample_tag = fp.replace("_results/output_","").replace("ABCD","").replace(".json","")[:-1]
        samples.append(sample_tag)

        region=data[regiontype][regionname]
        
        for b in ['TT', 'TL', 'LT', 'LL']:
            signalresults[b]['data'] += region[b]["data"]
            signalresults[b]['real']['sumweights'] += region[b]["real"]["sumweights"]
            signalresults[b]['jfp']['sumweights'] += region[b]["jfp"]["sumweights"]
            signalresults[b]['efp']['sumweights'] += region[b]["efp"]["sumweights"]
            signalresults[b]['other']['sumweights'] += region[b]["other"]["sumweights"]
            signalresults[b]['unclassified']['sumweights'] += region[b]["unclassified"]["sumweights"]

    return signalresults

In [13]:
sigsamples=["N2_200_N1_185",
            "N2_200_N1_190",
            "N2_200_N1_195",
            "N2_200_N1_197"]

def printregion(regiontype="SR", region="0L-mT-low", Run2=True):
    blindTT = (regiontype == "SR")
    
    #debugoutput=(regiontype=="VR")
    debugoutput = False

    if not debugoutput:
        print(f"{regiontype}-{region}")
        print(f"      ID    Isolation :   JFP,MC   JFP,DD     {sigsamples[0]}   {sigsamples[1]}   {sigsamples[2]}   {sigsamples[3]}")
        print("----------------------------------------------------------------------------------------------------------")
    
    for ID in ["mediumID","tightID"]:
        for Iso in ["looseIso", "tightIso", "tightCOIso", "hybridIso", "hybridCOIso", "noIso"]:
            if debugoutput: print(f"Results for {ID}, {Iso}:\n")
            
            totalresults,samples=getfakeestimate(regiontype,region,ID,Iso,Run2,False)
            
            N={}
            N_MC={}
            N_JFP_MC={}
            if debugoutput: print(f"      Data       MC      Real      EFP    Other      JFP")
            for b in ['TT', 'TL', 'LT', 'LL']:
                N[b] = totalresults[b]['data']-totalresults[b]['real']['sumweights']-totalresults[b]['efp']['sumweights']
                N_JFP_MC[b] = totalresults[b]['jfp']['sumweights'] + totalresults[b]['other']['sumweights'] 
                N_MC[b] = N_JFP_MC[b] + totalresults[b]['real']['sumweights'] + totalresults[b]['efp']['sumweights']
                if debugoutput:
                    print(f"{b}: {totalresults[b]['data'] if b != 'TT' or (not blindTT) else 0:6d}   {N_MC[b]:6.1f}    {totalresults[b]['real']['sumweights']:6.1f}   {totalresults[b]['efp']['sumweights']:6.1f}   {totalresults[b]['other']['sumweights']:6.1f}   {totalresults[b]['jfp']['sumweights']:6.1f}")
            if debugoutput: print('')
            
            if N['LL']>0:
                N_TT_bkg_DDjfp = N['TL']*N['LT']/N['LL']
            else:
                N_TT_bkg_DDjfp = 0
            
            if N_JFP_MC['LL']>0:
                N_TT_bkg_DDMCjfp = N_JFP_MC['TL']*N_JFP_MC['LT']/N_JFP_MC['LL']
            else:
                N_TT_bkg_DDMCjfp = 0
            
            N_TT_bkg_real = totalresults['TT']['real']['sumweights']
            N_TT_bkg_other = totalresults['TT']['other']['sumweights']
            N_TT_bkg_MCjfp = totalresults['TT']['jfp']['sumweights']
            N_TT_bkg_efp = totalresults['TT']['efp']['sumweights']
            N_TT_bkg_unclassified = totalresults['TT']['unclassified']['sumweights']
            
            N_TT_bkg_MC = N_TT_bkg_MCjfp + N_TT_bkg_real + N_TT_bkg_other + N_TT_bkg_efp
            
            N_TT_bkg_DD = N_TT_bkg_DDjfp + N_TT_bkg_real + N_TT_bkg_efp
            
            N_TT_bkg_DDMC = N_TT_bkg_DDMCjfp + N_TT_bkg_real + N_TT_bkg_efp
            
            if debugoutput: print(f"N_TT_bkg = ({N['TL']:.1f}*{N['LT']:.1f})/({N['LL']:.1f}) = {N_TT_bkg_DDjfp:.1f}")
            
            if debugoutput: print("")
    
            if debugoutput:
                if not blindTT:
                    print(f"Total data in TT region is {totalresults['TT']['data']:.1f}.")
                print(f"DD background prediction: {N_TT_bkg_real:5.1f} (real) + {N_TT_bkg_DDjfp:5.1f} (jfp+other) + {N_TT_bkg_efp:.1f} (efp) + {N_TT_bkg_unclassified:.1f} (unclassified) = {N_TT_bkg_DD:.1f}")
                print(f"MC background prediction: {N_TT_bkg_real:5.1f} (real) + {N_TT_bkg_MCjfp+N_TT_bkg_other:5.1f} (jfp+other) + {N_TT_bkg_efp:.1f} (efp) + {N_TT_bkg_unclassified:.1f} (unclassified) = {N_TT_bkg_MC:.1f}")    
                print(f"MC background closure   : {N_TT_bkg_real:5.1f} (real) + {N_TT_bkg_DDMCjfp:5.1f} (jfp+other) + {N_TT_bkg_efp:.1f} (efp) + {N_TT_bkg_unclassified:.1f} (unclassified) = {N_TT_bkg_DDMC:.1f}")        
    
    
            yields={}
            yieldsstring=""
            for sigsample in sigsamples:
                sigsampleresults=getsignalestimate(regiontype,region,ID,Iso,sigsample,Run2,False)
                #print(json.dumps(sigsampleresults,indent=4))
                yields[sigsample]=sigsampleresults['TT']['real']['sumweights']
                if debugoutput: print(f"Signal sample {sigsample} has {yields[sigsample]:.1f} events")
                yieldsstring += f"{yields[sigsample]:13.1f}   "
                
            if not debugoutput:
                print(f"{ID:10s} {Iso:11s}:   {N_TT_bkg_MCjfp+N_TT_bkg_other:6.1f}    {N_TT_bkg_DDjfp:5.1f}     {yieldsstring}")
            print("----------------------------------------------------------------------------------------------------------")

regiontype="SR"
region="0L-mT-low"

#regiontype="VR"
#region="0L-mT-mid"
#debugoutput=True

for region in ["0L-mT-low","0L-mT-mid","0L-mT-hgh"]:
    printregion(r"SR", region, True)
printregion(r"VR", "0L-mT-mid", True)

SR-0L-mT-low
      ID    Isolation :   JFP,MC   JFP,DD     N2_200_N1_185   N2_200_N1_190   N2_200_N1_195   N2_200_N1_197
----------------------------------------------------------------------------------------------------------
mediumID   looseIso   :      1.2     -1.4               5.4            12.2            12.0             8.4   
----------------------------------------------------------------------------------------------------------
mediumID   tightIso   :      1.1      1.3               4.5            10.8            12.4            11.7   
----------------------------------------------------------------------------------------------------------
mediumID   tightCOIso :      1.5      1.5               4.5            10.8            12.9            12.1   
----------------------------------------------------------------------------------------------------------
mediumID   hybridIso  :      1.1     -1.1               5.5            12.4            12.4            11.7   
-------

Quick function that will test closure for any single sample.

In [6]:
def sampleABCD(sample,debug=False,ID="tightID",Iso="hybridIso"):
    sresults=None
    if isinstance(sample,str):
        sname=f"ABCD_results/output_{sample}_ABCD_{ID}_{Iso}.json"
        sresults=load_json_file(sname)["VR"]["0L-mT-mid"]
        #print(json.dumps(results,indent=4,cls=NumpyEncoder))
    elif isinstance(sample,dict):
        sresults=sample
    else:
        print("Must provide either valid sample string or dictionary of results.")
        return None

    N_all={}
    for b in ['TT', 'TL', 'LT', 'LL']:
        N_all[b] = sresults[b]['real']['sumweights']+sresults[b]['jfp']['sumweights']+sresults[b]['efp']['sumweights']+sresults[b]['other']['sumweights']

    num_TL = (N_all['TL']-sresults['TL']['real']['sumweights']-sresults['TL']['efp']['sumweights'])
    num_LT = (N_all['LT']-sresults['LT']['real']['sumweights']-sresults['LT']['efp']['sumweights'])
    den_LL = (N_all['LL']-sresults['LL']['real']['sumweights']-sresults['LL']['efp']['sumweights'])
    N_TT_jfp_est = 0.
    if den_LL > 0:
        N_TT_jfp_est = num_TL*num_LT/den_LL

    if debug and den_LL > 0 and sresults['TT']['jfp']['sumweights']>0:
        print(f"{sample:60s} {sresults['TT']['real']['sumweights']:6.1f}  {N_TT_jfp_est:6.1f}  {sresults['TT']['jfp']['sumweights']:6.1f}   {sresults['TT']['other']['sumweights']:6.1f}  {sresults['TT']['efp']['sumweights']:6.1f}   {(N_TT_jfp_est)/(sresults['TT']['jfp']['sumweights']+sresults['TT']['other']['sumweights']):6.1f}")
    elif debug:
        print(f"{sample:60s} {sresults['TT']['real']['sumweights']:6.1f}  {N_TT_jfp_est:6.1f}  {sresults['TT']['jfp']['sumweights']:6.1f}   {sresults['TT']['other']['sumweights']:6.1f}  {sresults['TT']['efp']['sumweights']:6.1f}")
        
    return N_TT_jfp_est

In [7]:
ID="tightID"
Iso="hybridIso"
totalresults,samples=getfakeestimate(regiontype,region,ID,Iso,False,False)
print(f"{"Sample":60s} {"Prompt":6s}    {"ABCD":6s} {"MC JFPs":6s}  {"Other":6s}  {"EFP":6s}  {"ABCD/MC":6s}")
for s in sorted(samples):
    est=sampleABCD(s.replace("__tightID_hybridIs",""),True,ID,Iso)

Sample                                                       Prompt    ABCD   MC JFPs  Other   EFP     ABCD/MC
MGH7EG_tty_dec_mc23                                             0.0     0.0     0.0      0.0     0.0
MGPy8EG_tty_dec_mc23                                            0.0     0.0     0.0      0.0     0.0
PhPy8EG_A14_ttbar_hdamp258p75_SingleLep_mc23                    0.1     0.2     0.6      1.0     0.1      0.1
PhPy8EG_A14_ttbar_hdamp258p75_allhad_mc23                       0.0     0.0     0.0      0.0     0.0
PhPy8EG_A14_ttbar_hdamp258p75_dil_mc23                          0.0     0.0     0.1      0.0     0.0      0.3
PhPy8EG_tW_dyn_DR_incl_antitop_mc23                             0.0     0.1     0.1      0.0     0.0      1.1
PhPy8EG_tW_dyn_DR_incl_top_mc23                                 0.0     0.0     0.0      0.0     0.0
PhPy8EG_tb_lep_antitop_mc23                                     0.0     0.0     0.0      0.0     0.0
PhPy8EG_tb_lep_top_mc23                               

This seems to be working.  To do:
* Implement some other regions

In [8]:
Run2=True
totalresults,samples=getfakeestimate(regiontype,region,ID,Iso,Run2,False)
print(f"{"Sample":60s} {"Prompt":6s}    {"ABCD":6s} {"MC JFPs":6s}  {"Other":6s}  {"EFP":6s}  {"ABCD/MC":6s}")
for s in sorted(samples):
    est=sampleABCD(s.replace("__tightID_hybridIs",""),True,ID,Iso)

Sample                                                       Prompt    ABCD   MC JFPs  Other   EFP     ABCD/MC
MGPy8EG_tty_yfromdec_mc20                                       0.0     0.0     0.0      0.0     0.0
PhPy8EG_A14_tchan_BW50_had_antitop_mc20                         0.0     0.0     0.0      0.0     0.0
PhPy8EG_A14_tchan_BW50_had_top_mc20                             0.0     0.0     0.0      0.0     0.0
PhPy8EG_A14_tchan_BW50_lept_antitop_mc20                        0.0     0.1     0.0      0.0     0.0
PhPy8EG_A14_tchan_BW50_lept_top_mc20                            0.0     0.3     0.2      0.1     0.1      1.3
PhPy8EG_tW_dyn_DR_incl_antitop_mc20                             0.0     0.3     0.0      0.3     0.0
PhPy8EG_tW_dyn_DR_incl_top_mc20                                 0.0     0.5     0.0      0.1     0.0
PowhegPythia8EvtGen_A14_singletop_schan_lept_antitop_mc20       0.0     0.0     0.0      0.0     0.0
PowhegPythia8EvtGen_A14_singletop_schan_lept_top_mc20           0.0     